In [28]:
import scipy.integrate as integrate 
import math
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.integrate as integrate 
import cmath 
from scipy import optimize
from scipy.misc import derivative

In [27]:
# Begin with defining everything
# These are the coupling constants of the lagrangian, and the masses of the particles

eps = 0.0000001

#yukawa coupling constant of fermion f 
#(for the top quark)
y_f = 0.99                                          


#EW - sector coupling constants g (SU(2)_L) and g_p (U(1)_Y) 
g = 0.65
g_p = 0.36

# Effective mass of fermion f acquired through yukawa coupling w/ the Higgs
def m_f(phi):
    return y_f**2/2*phi**2 + eps

# W and Z boson masses
def m_W(phi):
    return (g**2/4)*phi**2 + eps

def m_Z(phi):
    return (g**2 + g_p**2)*phi**2/4 + eps

# Higgs and Goldstone masses
def m_h(phi):                                               
    return -3*lam*phi**2 - mu + (15/4)*phi**4/Lam**2 + eps

    #m_h 0 at h_min = (2/5)*(lam/c6)*(1 - np.sqrt(1-5/3*c6*(mu**2)/lam**2))

def m_g(phi):
    return -lam*phi**2 - mu + (3/4)*phi**4/Lam**2 + eps


In [29]:
# different parts of the 1-loop effective potential calculated in the ms-scheme.
# separating the contribution coming from gauge bosons and other particles in the loop correction. 
# obs! input for the first order corrections are the squared masses, i.e. m**2 and not m.  

def v_tree(phi):
    return -mu/2*phi**2 - lam/4*phi**4 + 1/(8*Lam**2)*phi**6

def v_loop_g(m):
    #Coleman-Weinberg potential for gauge bosons as a function of mass squared.
    scale = 246
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 5/6)

def v_loop(m):
    #Coleman-Weinberg potential for particles as a function of mass squared.
    scale = 246
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 3/2)


def v_1(phi):
    return np.real(v_loop(m_h(phi)) + 3*v_loop(m_g(phi)) - 12*v_loop(m_f(phi))
             + 3*v_loop_g(m_Z(phi)) + 6*v_loop_g(m_W(phi)))

#this is the temperature independent part of the effective potential
def v_eff_0(phi):
    return np.real(v_tree(phi) + v_1(phi))


#defining high T expansion of the bosonic (j_b) and fermionic (j_f) thermal functions
def j_b(y):
    a_b = np.pi**2*np.exp(3/2 - 2*0.5572)
    return (np.pi**2/12)*y**2 - np.pi/6*y**3 - (1/32)*y**4*cmath.log(y**2/a_b)

def j_f(y):
    a_f = 16*np.pi**2*np.exp(3/2 - 2*0.5572)
    return -(np.pi**2/24)*y**2 - (1/32)*y**4*cmath.log(y**2/a_f)

def v_eff_T(phi,T):
    return T**4/(2*np.pi**2)*(j_b(cmath.sqrt(m_h(phi))/T) + 3*j_b(cmath.sqrt(m_g(phi))/T) + 3*j_b(cmath.sqrt(m_Z(phi))/T)
                             + 6*j_b(cmath.sqrt(m_W(phi))/T) - 12*j_f(cmath.sqrt(m_f(phi))/T))


In [30]:
# Loading the parameter values to be used in the GW spectrum calculations
# the parameters are mu, lambda and c6 which go into the tree-level effective potential. They have been chosen
# to give the correct higgs mass (125) and vaccum (246) for the 1-loop effective potential at zero temperature.

# The parameters are saved in a list, and need to be reshaped such that each row is giving a set of 
# mu, lambda, c6 that together satisfy the conditions mentioned above.

parameters = np.loadtxt('parameters.txt').reshape(-1,3)
#print(parameters)

In [31]:
def find_t_crit(t_span, tolerance):
    T = np.mean(t_span)
    
    def v_eff_T(phi):
        return v_eff(phi,T)
    
    symm_vac = optimize.fmin(v_eff_T, 0, disp = 0)[0]
    broken_vac = optimize.fmin(v_eff_T, 300, disp = 0)[0]
    
    
    min1 = v_eff_T(symm_vac)
    min2 = v_eff_T(broken_vac)
    
    if np.abs(symm_vac - broken_vac) <= tolerance:
        #this is condition for T being too high and only in symmetric phase, Tis to high.
        return find_t_crit([t_span[0], np.mean(t_span)], tolerance)
    
    elif np.abs(min2 - min1) <= tolerance:
        #condition for being at the critical temperature
        return T, broken_vac
    
    elif min2 - min1 > tolerance:
        #the broken phase is higher up than the symmetric one, Temperature is to high
        return find_t_crit([t_span[0], np.mean(t_span)],tolerance)
    else:
        return find_t_crit([np.mean(t_span), t_span[1]],tolerance) 
    

In [32]:
# Overshoot-undershoot implementation to find the bounce solution. Works in the same way as finding the critical temp
# algorithm get's stuck somtetimes due to np.mean not being sensitive enough(?).

def over_undershoot(init_span, tol, max_i, iter = None):
    
    if iter is None:
        iter = 0
        
    iter +=1    
    
    if iter == max_i:
        
        print('arrived at max iterations')
        print('final span: ', init_span)
        
    else:
        t_span = np.array([eps, 1])
        times  = np.linspace(t_span[0], t_span[1], 100) 
        
        
        init = np.mean(init_span)
        
        
        y0 = np.array([init, 0])
    
        soln = solve_ivp(f, t_span, y0, t_eval = times)
        #print(soln.message)   
    
    
        t = soln.t
        phi_b = soln.y[0] 
    
    
        if np.abs(np.min(phi_b) - symm_vac) <= tol:
            # Here we have the bounce solution. r_max is the bubble radius, smallest value of the solution which satisfies
            # the bounce condition and run the solution again for that 
            index_r = np.where(np.abs(phi_b - symm_vac) <= tol)
            r_max = t[np.min(index_r)]
            
            t_span = np.array([eps, r_max])
            times  = np.linspace(t_span[0], t_span[1], 100) 
            y0 = np.array([init, 0])
    
            soln = solve_ivp(f, t_span, y0, t_eval = times)
            
            r = soln.t
            phi_b = soln.y[0] 
            
            #print('Found it! y0=', init)
            return phi_b, t, soln.y[1], init, r_max
    
        elif phi_b[-1] < symm_vac - tol or phi_b[-1] > broken_vac:
            #condition for rolling down to the left towards phi = -inf &
            #condition for rolling down to the right towards phi = +inf  
            #This means we are too far high up in the span and initial condition is to high.
        
            #print('iteration: ', iter)
            #print('y0 = ', init)
            #print('overshoot')
            #print('final value of the solution:', phi_b[-1])
            #print('\n')
            return over_undershoot([init_span[0], init], tol, max_i, iter)
        
        else:  #phi_b[-1] > symm_vac + tol and phi_b[-1] < init:
            # All other scenarios are undershoot.
            # The initial value is to low need to look higher up in the interval for the correct initial value
            
            #print('iteration: ', iter)
            #print('y0 = ',  init)
            #print('undershoot')
            #print('smallest value of the solution:' , np.min(phi_b))
            #print('And it ended at', phi_b[-1])
            #print('\n')
            return over_undershoot([init, init_span[1]],tol, max_i, iter)

In [34]:
# for each set of parameters we now calculate the latent heat, invers nucleation time and nucelation temperature.
 
GWP = []
for row in parameters:
    
    mu, lam, Lam = row
    
    print('\u03BC, \u03BB, \u039B = ',mu, lam, Lam)
    def v_eff(phi,T):
        return np.real(v_eff_0(phi) + v_eff_T(phi,T)) - np.real(v_eff_0(0) + v_eff_T(0,T)) 
    
    
    a = -6*lam + 9/4*g**2 + 3/4*g_p**2 + 3*y_f**2
    
    
    T_min = np.real(np.sqrt(12/a)*cmath.sqrt(mu))
    
    t_span = [1.01*T_min, 300]
    tolerance = 1
    
    t_c , phi_c = find_t_crit(t_span, tolerance)
    print('critical temperature:', t_c)
    print('strength of phase transition', phi_c/t_c)
    print('\n')
        
    max_iter = 100
    i = 0
    
    #initial guess for span of nucleation temperature and do a binary search in while loop    
    tn_span = [T_min, t_c] 
    
    while i <= max_iter:
        tn = (tn_span[0] + tn_span[1])/2
        
        print('T =', tn)
        i += 1
        

        def v_eff_tn(phi):
            return np.vectorize(v_eff)(phi,tn)


        #derivative of v_eff at nucleation temperature
        def dv_eff(phi):
            return derivative(v_eff_tn, phi, dx = 0.00001, order=7)
        
        def f(t,y):    
            A = y[0]
            B = y[1]
    
            dA_dt = B
            dB_dt = dv_eff(A) - (2/t)*B                                
            return np.array([dA_dt, dB_dt])
        
        symm_vac = 0

        broken_vac = optimize.fmin(v_eff_tn, 300, disp = 0)[0]
        tol = 2

                
        init_span = np.array([symm_vac, broken_vac])     
                                                      
        phi_b , r , phi_dot, ic, r_max = over_undershoot(init_span, tol, 100)

        
        v_bounce = np.vectorize(v_eff)(phi_b,tn)


        #R is the interval over which we want to integrate the functional over to get the bounce action.
        #L is the euclidean lagrangian density to be integrated.  

        R = np.linspace(0, r_max, len(phi_b))
        L = ((1/2)*(phi_dot)**2 + v_bounce)

        
        
        s3 = 4*np.pi*np.trapz((R**2)*L, R)
        
        print('bounce action over nucleation temperature', s3/tn)
        print('\n')
        
        if 150 >= s3/tn >= 130:
            break
        elif s3/tn > 150:
            #T is too high need to lower
            tn_span = [tn_span[0], tn]
        else:
            tn_span = [tn, tn_span[1]]
    
    
    g_star = 106.95
    rho_rad = np.pi**2*g_star*(tn**4)/30
    delta_v = v_eff(symm_vac, tn) - v_eff(broken_vac,tn)
    delta_dv = dv_eff(symm_vac) - dv_eff(broken_vac)

    alpha = (1/rho_rad)*(delta_v - (1/4)*delta_dv)
    
    def dv_dT(phi,T):
        # Fourth order finite difference scheme for approximation of derivative off effective action along T
        h = 1.4901161193847656e-8
    
        return np.real(-v_eff(phi, T + 2*h) + 8*v_eff(phi, T + h) - 8*v_eff(phi,T - h)
                      + v_eff(phi, T - 2*h))/(12*h)

    dv = np.vectorize(dv_dT)(phi_b,tn)
    
    ds3_dT = 4*np.pi*np.trapz(R**2*dv, R)

    beta = ds3_dT - s3/tn
    
    print('\u03B1, \u03B2, T =', alpha, beta, tn)
    print('\n')
    GWP.append([alpha, beta, tn])

μ, λ, Λ =  181.11189133118546 0.05224133191889968 634.4481474884878
critical temperature: 109.45231214181845
strength of phase transition 2.480364838827524


T = 66.88590265948912
bounce action over nucleation temperature 93.10049233286966


T = 88.16910740065379
bounce action over nucleation temperature 203.69207288894853


T = 77.52750503007145
bounce action over nucleation temperature 125.56515037769331


T = 82.84830621536261
bounce action over nucleation temperature 154.69182415051932


T = 80.18790562271704
bounce action over nucleation temperature 138.39048165036635


α, β, T = 0.040106936938423675 438.0082265225701 80.18790562271704


μ, λ, Λ =  860.5139927716356 0.039738664765014683 634.6327876197859
critical temperature: 112.09604882050593
strength of phase transition 2.351278170712213


T = 82.28668967101568
bounce action over nucleation temperature 90.35227872540294


T = 97.1913692457608
bounce action over nucleation temperature 230.9043493180026


T = 89.73902945838825
bo

TypeError: cannot unpack non-iterable NoneType object

In [35]:
print(GWP)

[[0.040106936938423675, 438.0082265225701, 80.18790562271704], [0.02108180648993731, 712.1065821679747, 89.73902945838825], [0.012015793896509095, 1328.004971266169, 92.49585760575351], [0.010868979460093941, 1323.6511029809749, 100.00267044390631], [0.008069331119557864, 1719.7637131674526, 103.99811238503376]]


In [36]:
GWP_file = open('GWparameters.txt', 'w')

for row in GWP:
    np.savetxt(GWP_file, row)
    
GWP_file.close()

og_array = np.loadtxt('GWparameters.txt').reshape(-1,3)
print(og_array)

[[4.01069369e-02 4.38008227e+02 8.01879056e+01]
 [2.10818065e-02 7.12106582e+02 8.97390295e+01]
 [1.20157939e-02 1.32800497e+03 9.24958576e+01]
 [1.08689795e-02 1.32365110e+03 1.00002670e+02]
 [8.06933112e-03 1.71976371e+03 1.03998112e+02]]
